In [ ]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
WORKSPACE_NAME=''
SUBSCRIPTION_ID=''
RESOURCE_GROUP=''
LOCATION=''

In [ ]:
from azureml.core import Workspace, Experiment, Run
from azureml.exceptions import WorkspaceException

try:
    ws = Workspace.create(
        subscription_id=SUBSCRIPTION_ID, 
        resource_group=RESOURCE_GROUP,
        name=WORKSPACE_NAME,
        location=LOCATION
    )
except WorkspaceException:
    ws = Workspace.get(
        subscription_id=SUBSCRIPTION_ID, 
        resource_group=RESOURCE_GROUP,
        name=WORKSPACE_NAME
    )

In [ ]:
# show workspace details
ws.get_details()

In [ ]:
experiment = Experiment(workspace = ws, 
                        name = "automl-experiment")

# Training

Train your model in any way...

# Register ML Model

In [ ]:
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl",
                       model_name = "sklearn_regression_model.pkl",
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace = ws)

# Deployment

## Create a scoring script

In [ ]:
%%writefile score.py
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model
import json

def init():
    global model
    model_path = Model.get_model_path('sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any datatype if it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

## Create the environment YAML file

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## Create a container image for a trained model

In [ ]:
from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "diabetes", 'type': "regression"},
                                 description = "Image with ridge regression model")
image = Image.create(name = "myimage1",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

## Deploy the model as a web service

In [ ]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                          memory_gb = 1, 
                                          tags = {"data": "mnist", "type": "classification"}, 
                                          description = 'Handwriting recognition')

In [ ]:
from azureml.core.webservice import Webservice
service_name = 'aci-mnist-13'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)
service.wait_for_deployment(show_output = True)

## Score data with the deployed model

In [ ]:
import json
test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})
test_sample = bytes(test_sample,encoding = 'utf8')
prediction = service.run(input_data = test_sample)
print(prediction)